### Huggingfaceのよくわかってないところを検証していくnotebook

#### hidden_sizeはAutoConfigで統一できそう

In [1]:
from transformers import AutoConfig, AutoModel

In [8]:
cfg = AutoConfig.from_pretrained("cl-tohoku/bert-base-japanese")
cfg.hidden_size

768

In [9]:
cfg = AutoConfig.from_pretrained("rinna/japanese-roberta-base")
cfg.hidden_size

768

In [13]:
cfg = AutoConfig.from_pretrained("ganchengguang/Roformer-base-japanese")
cfg

RobertaConfig {
  "_name_or_path": "ganchengguang/Roformer-base-japanese",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 768,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.22.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32003
}

In [11]:
cfg = AutoConfig.from_pretrained("cl-tohoku/bert-large-japanese")
cfg.hidden_size

1024

In [15]:
cfg = AutoConfig.from_pretrained("nlp-waseda/roberta-large-japanese-seq512")
cfg.hidden_size

1024

In [16]:
cfg = AutoConfig.from_pretrained("rinna/japanese-gpt-1b")
cfg.hidden_size

2048

In [17]:
cfg = AutoConfig.from_pretrained("rinna/japanese-gpt2-medium")
cfg.hidden_size

1024

In [18]:
cfg = AutoConfig.from_pretrained("xlm-roberta-large")
cfg.hidden_size

1024

#### AutoModelとAutoModelfor##LMの違い
* AutoModelは完全なBare
* 語尾つきはタスクに合わせてヘッダが追加されている(らしい)
https://discuss.huggingface.co/t/difference-between-automodel-and-automodelforlm/5967

* ヘッダを見た感じ、やっぱり活性化関数いるんじゃない？

In [19]:
AutoModel.from_pretrained("rinna/japanese-roberta-base")

Some weights of the model checkpoint at rinna/japanese-roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at rinna/japanese-roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to b

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=3)
    (position_embeddings): Embedding(514, 768, padding_idx=3)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

In [21]:
from transformers import AutoModelForCausalLM
AutoModelForCausalLM.from_pretrained("rinna/japanese-roberta-base")

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


RobertaForCausalLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=3)
      (position_embeddings): Embedding(514, 768, padding_idx=3)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor